In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15069, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15069 (delta 1), reused 3 (delta 0), pack-reused 15056
Receiving objects: 100% (15069/15069), 13.44 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (10235/10235), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [5]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!unzip -q "/content/drive/MyDrive/비전/testdata_yolo/test.zip" -d /content/darknet/data/

드라이브에서 웨이트파일 및 테스트 데이터셋 업로드

테스트 데이터셋 업로드 -> data/ 에다가

========로컬에서 =============

data/obj.data -> 업로드 및 경로수정

data/obj.names-> 업로드 및 클래스 수정

cfg/yolov4-obj.cfg 파일 업로드 및 파일이름에서 txt제거

data/test.txt 업로드




In [8]:
#weight
!unzip -q "/content/drive/MyDrive/비전/yoloweight/yolov4-obj_best.zip" -d /content/darknet/

In [ ]:
!pwd

/content/darknet


In [9]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' cp.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' cp.cfg
%cd ..

/content/darknet/cfg
/content/darknet


In [14]:
!./darknet detector test data/obj.data cfg/cp.cfg cp_last.weights -dont_show -ext_output <data/test.txt> result.txt

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128 
  10 conv 

image 구조체의 w,h가 load_image의 파라미터로 받은 w,h와 일치하지 않는다면, 일치하도록 image 구조체의 image 크기를 resize합니다. -> 그런데 weight 파일을 로드한 다음에 Cannot load image data/test/KakaoTalk_20210523_091607839.jpg 에러가 남

출처: https://dhhwang89.tistory.com/118 [Martin]

In [ ]:
!git clone https://github.com/Cartucho/mAP.git


fatal: destination path 'mAP' already exists and is not an empty directory.


In [ ]:
!cp result.txt mAP/scripts/extra 

In [ ]:
%cd mAP/scripts/extra 

/content/darknet/mAP/scripts/extra


In [ ]:
!pwd

/content/darknet/mAP/scripts/extra


In [ ]:
!ls

class_list.txt		     convert_gt_yolo.py      README.md
convert_dr_darkflow_json.py  convert_keras-yolo3.py  result.txt
convert_dr_yolo.py	     find_class.py
convert_gt_xml.py	     intersect-gt-and-dr.py


In [ ]:
!python convert_dr_yolo.py